### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [1]:
#for interactive plots

# %matplotlib
# %matplotlib inline
# %matplotlib notebook

In [2]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [106]:
class Particles:
    def __init__(self,x,v,m):
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    print("dx for i=", i, dx)
                    drsqr=np.sum(dx**2)
                    print("drsqr", drsqr)
                    
#                     drsqr[drsqr<soft**2]=soft**2
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                        print("soft")
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    print("r3, f?", r3, dx*self.m[j]*r3)
                    
#                   r=np.sqrt(drsqr)
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        print(self.f)
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    print("dx for i=", i, dx)
                    drsqr=np.sum(dx**2)
                    print("drsqr", drsqr)
                    
#                     drsqr[drsqr<soft**2]=soft**2
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                        print("soft")
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    print("r3, f?", r3, dx*self.m[j]*r3)
                    
#                   r=np.sqrt(drsqr)
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        print(self.f)

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn

    
    def take_step_simple(self,dt): #this is bad, but it does what it should - no dt softening here
        self.get_forces()
        self.x=self.x+dt*self.v
        self.v=self.v+dt*self.f
        
    def take_step_better(self, dt, soft=0.01): #works, is decent - also no dt softening
        xx=self.x+0.5*self.v*dt #first half step
        #gets forces for centerpoint of step
        self.get_forces_2(xx, soft)
        f=self.f
        
        #gets velocity for centerpoint of step
        vv=self.v+0.5*dt*f
        self.x=self.x+dt*vv
        self.v=self.v+dt*f

    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        

In [103]:
n=2
#for random starting
# x=np.random.randn(n,2)
# v=np.random.randn(n,2)

x=np.zeros([n,2])
v=np.zeros([n,2])
x[0,0]=1
x[1,0]=-1
v[0,1]=0.25
v[1,1]=-0.25

# x[2,0]=-0.5
# v[2,1]=0

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig, ax = plt.subplots()      

# for k in range(10):
#     for l in range(10):
        
for k in range(100):
    for l in range(20):
#         parts.take_step_simple(dt)
#         parts.take_step_better(dt)
        parts.take_step_leapfrog(dt)
    
    print('step ',k)
#     print(parts.x)
#     f,pot=get_forces(x,do_pot=True)
#     kin=np.sum(v**2)/2
    #print(kin,pot,kin-pot/2)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], color=colors[n_i])
#     plt.axis([-2,2,-2,2])
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)

print()

dx for i= 0 [-2.  0.]
drsqr 4.0
r3, f? 0.125 [-0.25  0.  ]
dx for i= 1 [2. 0.]
drsqr 4.0
r3, f? 0.125 [0.25 0.  ]
dx for i= 0 [-1.99995 -0.005  ]
drsqr 3.9998250024999997
r3, f? 0.125008203456431 [-0.25001016 -0.00062504]
dx for i= 1 [1.99995 0.005  ]
drsqr 3.9998250024999997
r3, f? 0.125008203456431 [0.25001016 0.00062504]
dx for i= 0 [-1.99985    -0.00999987]
drsqr 3.999500011875259
r3, f? 0.12502344060581674 [-0.25002813 -0.00125022]
dx for i= 1 [1.99985    0.00999987]
drsqr 3.999500011875259
r3, f? 0.12502344060581674 [0.25002813 0.00125022]
dx for i= 0 [-1.99969999 -0.0149995 ]
drsqr 3.9990250362519006
r3, f? 0.12504571535378817 [-0.25005392 -0.00187562]
dx for i= 1 [1.99969999 0.0149995 ]
drsqr 3.9990250362519006
r3, f? 0.12504571535378817 [0.25005392 0.00187562]
dx for i= 0 [-1.99949997 -0.01999875]
drsqr 3.9984000875080836
r3, f? 0.12507503341146176 [-0.25008753 -0.00250134]
dx for i= 1 [1.99949997 0.01999875]
drsqr 3.9984000875080836
r3, f? 0.12507503341146176 [0.25008753 0.00

drsqr 3.7350424036681793
r3, f? 0.1385341023421825 [-0.26469042 -0.04025898]
dx for i= 1 [1.91065171 0.29060702]
drsqr 3.7350424036681793
r3, f? 0.1385341023421825 [0.26469042 0.04025898]
step  2
dx for i= 0 [-1.90759287 -0.29537559]
drsqr 3.726157286182931
r3, f? 0.1390299047851478 [-0.26521245 -0.04106604]
dx for i= 1 [1.90759287 0.29537559]
drsqr 3.726157286182931
r3, f? 0.1390299047851478 [0.26521245 0.04106604]
dx for i= 0 [-1.90448098 -0.30013594]
drsqr 3.7171293904280907
r3, f? 0.1395367109048722 [-0.26574501 -0.04187998]
dx for i= 1 [1.90448098 0.30013594]
drsqr 3.7171293904280907
r3, f? 0.1395367109048722 [0.26574501 0.04187998]
dx for i= 0 [-1.90131594 -0.30488793]
drsqr 3.707958967901891
r3, f? 0.14005467838643623 [-0.26628819 -0.04270098]
dx for i= 1 [1.90131594 0.30488793]
drsqr 3.707958967901891
r3, f? 0.14005467838643623 [0.26628819 0.04270098]
dx for i= 0 [-1.89809765 -0.30963137]
drsqr 3.698646275012898
r3, f? 0.1405839694177189 [-0.2668421  -0.04352921]
dx for i= 1 [1

dx for i= 0 [-1.62189498 -0.55960874]
drsqr 2.9437052582097016
r3, f? 0.19799695649637966 [-0.32113027 -0.11080083]
dx for i= 1 [1.62189498 0.55960874]
drsqr 2.9437052582097016
r3, f? 0.19799695649637966 [0.32113027 0.11080083]
dx for i= 0 [-1.61531457 -0.56350391]
drsqr 2.9267778175720967
r3, f? 0.19971715360269301 [-0.32260603 -0.1125414 ]
dx for i= 1 [1.61531457 0.56350391]
drsqr 2.9267778175720967
r3, f? 0.19971715360269301 [0.32260603 0.1125414 ]
dx for i= 0 [-1.60866964 -0.56737657]
drsqr 2.9097341923845303
r3, f? 0.2014744708244422 [-0.32410587 -0.11431189]
dx for i= 1 [1.60866964 0.56737657]
drsqr 2.9097341923845303
r3, f? 0.2014744708244422 [0.32410587 0.11431189]
dx for i= 0 [-1.6019599  -0.57122636]
drsqr 2.892575066464333
r3, f? 0.20326988548381805 [-0.3256302  -0.11611312]
dx for i= 1 [1.6019599  0.57122636]
drsqr 2.892575066464333
r3, f? 0.20326988548381805 [0.3256302  0.11611312]
dx for i= 0 [-1.59518502 -0.57505293]
drsqr 2.8753011343597645
r3, f? 0.20510440911869685 [-

dx for i= 0 [-1.3048739  -0.69366469]
drsqr 2.1838666042597428
r3, f? 0.3098566629315993 [-0.40432387 -0.21493663]
dx for i= 1 [1.3048739  0.69366469]
drsqr 2.1838666042597428
r3, f? 0.3098566629315993 [0.40432387 0.21493663]
dx for i= 0 [-1.29542062 -0.69630295]
drsqr 2.1629523747380417
r3, f? 0.31436165357530055 [-0.40723057 -0.21889095]
dx for i= 1 [1.29542062 0.69630295]
drsqr 2.1629523747380417
r3, f? 0.31436165357530055 [0.40723057 0.21889095]
dx for i= 0 [-1.28588589 -0.69889743]
drsqr 2.1419601328301443
r3, f? 0.3189943016278728 [-0.41019027 -0.2229443 ]
dx for i= 1 [1.28588589 0.69889743]
drsqr 2.1419601328301443
r3, f? 0.3189943016278728 [0.41019027 0.2229443 ]
dx for i= 0 [-1.27626912 -0.70144731]
drsqr 2.120891208209653
r3, f? 0.32375941918976153 [-0.41320415 -0.22710018]
dx for i= 1 [1.27626912 0.70144731]
drsqr 2.120891208209653
r3, f? 0.32375941918976153 [0.41320415 0.22710018]
dx for i= 0 [-1.26656971 -0.70395178]
drsqr 2.099746955210631
r3, f? 0.3286620499586146 [-0.41

drsqr 1.393043511978981
r3, f? 0.6082091887704631 [0.55242302 0.45841166]
dx for i= 0 [-0.89542602 -0.75405225]
drsqr 1.3703825440661557
r3, f? 0.6233576205183418 [-0.55817063 -0.47004422]
dx for i= 1 [0.89542602 0.75405225]
drsqr 1.3703825440661557
r3, f? 0.6233576205183418 [0.55817063 0.47004422]
dx for i= 0 [-0.8824624  -0.75430325]
drsqr 1.3477132915126113
r3, f? 0.639151382611042 [-0.56402706 -0.48211397]
dx for i= 1 [0.8824624  0.75430325]
drsqr 1.3477132915126113
r3, f? 0.639151382611042 [0.56402706 0.48211397]
dx for i= 0 [-0.86938598 -0.75445784]
drsqr 1.3250386167722625
r3, f? 0.655627553146843 [-0.5699934  -0.49464335]
dx for i= 1 [0.86938598 0.75445784]
drsqr 1.3250386167722625
r3, f? 0.655627553146843 [0.5699934  0.49464335]
step  9
dx for i= 0 [-0.85619556 -0.75451349]
drsqr 1.3023614561749768
r3, f? 0.6728258954073598 [-0.57607055 -0.50765622]
dx for i= 1 [0.85619556 0.75451349]
drsqr 1.3023614561749768
r3, f? 0.6728258954073598 [0.57607055 0.50765622]
dx for i= 0 [-0.84

...

dx for i= 0 [-1.5243925  -0.60461156]
drsqr 2.689327631796307
r3, f? 0.22674328216328468 [-0.34564576 -0.13709161]
dx for i= 1 [1.5243925  0.60461156]
drsqr 2.689327631796307
r3, f? 0.22674328216328468 [0.34564576 0.13709161]
dx for i= 0 [-1.51687346 -0.60818932]
drsqr 2.6707993377820305
r3, f? 0.2291068689657692 [-0.34752613 -0.13934035]
dx for i= 1 [1.51687346 0.60818932]
drsqr 2.6707993377820305
r3, f? 0.2291068689657692 [0.34752613 0.13934035]
dx for i= 0 [-1.50928491 -0.6117392 ]
drsqr 2.652165808352724
r3, f? 0.23152558607550366 [-0.34943807 -0.14163328]
dx for i= 1 [1.50928491 0.6117392 ]
drsqr 2.652165808352724
r3, f? 0.23152558607550366 [0.34943807 0.14163328]
dx for i= 0 [-1.50162648 -0.61526076]
drsqr 2.6334279018874946
r3, f? 0.23400107411408802 [-0.35138221 -0.14397168]
dx for i= 1 [1.50162648 0.61526076]
drsqr 2.6334279018874946
r3, f? 0.23400107411408802 [0.35138221 0.14397168]
dx for i= 0 [-1.49389777 -0.61875353]
drsqr 2.614586490745583
r3, f? 0.23653503726322744 [-0.3

dx for i= 0 [-0.65731395 -0.7388119 ]
drsqr 0.9779046504874651
r3, f? 1.0340826054728056 [-0.67971692 -0.76399253]
dx for i= 1 [0.65731395 0.7388119 ]
drsqr 0.9779046504874651
r3, f? 1.0340826054728056 [0.67971692 0.76399253]
dx for i= 0 [-0.64219298 -0.73702956]
drsqr 0.9556244073132616
r3, f? 1.0704568112421875 [-0.68743985 -0.78895832]
dx for i= 1 [0.64219298 0.73702956]
drsqr 0.9556244073132616
r3, f? 1.0704568112421875 [0.68743985 0.78895832]
dx for i= 0 [-0.62693453 -0.73508944]
drsqr 0.9334033872637655
r3, f? 1.1089090839292468 [-0.69521339 -0.81514736]
dx for i= 1 [0.62693453 0.73508944]
drsqr 0.9334033872637655
r3, f? 1.1089090839292468 [0.69521339 0.81514736]
dx for i= 0 [-0.61153703 -0.73298629]
drsqr 0.9112464342407844
r3, f? 1.1495986288501405 [-0.70302213 -0.84264003]
dx for i= 1 [0.61153703 0.73298629]
drsqr 0.9112464342407844
r3, f? 1.1495986288501405 [0.70302213 0.84264003]
dx for i= 0 [-0.59599892 -0.73071461]
drsqr 0.8891585536061434
r3, f? 1.1926999030144818 [-0.710

drsqr 0.1948935554087986
r3, f? 11.622613135759481 [-0.96059859 -5.04028499]
dx for i= 0 [0.06306618 0.4519033 ]
drsqr 0.20819393537312092
r3, f? 10.526836757458945 [0.66388743 4.75711226]
dx for i= 1 [-0.06306618 -0.4519033 ]
drsqr 0.20819393537312092
r3, f? 10.526836757458945 [-0.66388743 -4.75711226]
dx for i= 0 [0.04335048 0.46919323]
drsqr 0.2220215544834847
r3, f? 9.558886203181299 [0.41438235 4.48496472]
dx for i= 1 [-0.04335048 -0.46919323]
drsqr 0.2220215544834847
r3, f? 9.558886203181299 [-0.41438235 -4.48496472]
dx for i= 0 [0.02355191 0.48558617]
drsqr 0.23634862486627314
r3, f? 8.70302799016398 [0.20497291 4.22607007]
dx for i= 1 [-0.02355191 -0.48558617]
drsqr 0.23634862486627314
r3, f? 8.70302799016398 [-0.20497291 -4.22607007]
dx for i= 0 [0.00371234 0.5011339 ]
drsqr 0.2511489683036315
r3, f? 7.945164662124301 [0.02949513 3.98159136]
dx for i= 1 [-0.00371234 -0.5011339 ]
drsqr 0.2511489683036315
r3, f? 7.945164662124301 [-0.02949513 -3.98159136]
dx for i= 0 [-0.0161331

dx for i= 0 [-1.00905561  0.75890742]
drsqr 1.5941336999608915
r3, f? 0.4968358044567007 [-0.50133496  0.37705238]
dx for i= 1 [ 1.00905561 -0.75890742]
drsqr 1.5941336999608915
r3, f? 0.4968358044567007 [ 0.50133496 -0.37705238]
dx for i= 0 [-1.02087321  0.75788514]
drsqr 1.616572004121125
r3, f? 0.486527530224772 [-0.49668292  0.36873199]
dx for i= 1 [ 1.02087321 -0.75788514]
drsqr 1.616572004121125
r3, f? 0.486527530224772 [ 0.49668292 -0.36873199]
dx for i= 0 [-1.03259148  0.75678912]
drsqr 1.6389749252777697
r3, f? 0.47658628033255723 [-0.49211893  0.36067531]
dx for i= 1 [ 1.03259148 -0.75678912]
drsqr 1.6389749252777697
r3, f? 0.47658628033255723 [ 0.49211893 -0.36067531]
dx for i= 0 [-1.04421132  0.75562096]
drsqr 1.6613403055066631
r3, f? 0.46699485695758536 [-0.48764132  0.3528711 ]
dx for i= 1 [ 1.04421132 -0.75562096]
drsqr 1.6613403055066631
r3, f? 0.46699485695758536 [ 0.48764132 -0.3528711 ]
dx for i= 0 [-1.05573363  0.75438222]
drsqr 1.68366603419497
r3, f? 0.4577370580

KeyboardInterrupt: 

### Part 1: 
Using this code, show that a single particle starting at rest remains motionless. 

In [107]:
n=1
#single particle at rest
x=np.random.randn(n,2)
v=np.random.randn(n,2)*0

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig, ax = plt.subplots()      

for k in range(1000):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
#     print('step ',k)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], color=colors[n_i])
        
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)


KeyboardInterrupt: 

### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [108]:
n=2
#two particles in circular orbit

x=np.zeros([n,2])
v=np.zeros([n,2])
x[0,0]=1
x[1,0]=-1
v[0,1]=0.25
v[1,1]=-0.25

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig, ax = plt.subplots()      

        
for k in range(300):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], color=colors[n_i])
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)

step  0
step  1
step  2
step  3
step  4
step  5
step  6
step  7
step  8
step  9
step  10
step  11
step  12
step  13
step  14
step  15
step  16
step  17
step  18
step  19
step  20
step  21
step  22
step  23
step  24
step  25
step  26
step  27
step  28
step  29
step  30
step  31


KeyboardInterrupt: 

step  32
step  33
step  34
step  35
step  36
step  37
step  38
step  39
step  40
step  41
step  42
step  43
step  44
step  45
step  46


KeyboardInterrupt: 

step  47


KeyboardInterrupt: 

step  48
step  49
step  50
step  51
step  52
step  53
step  54
step  55
step  56
step  57
step  58
step  59
step  60
step  61
step  62
step  63
step  64
step  65
step  66
step  67
step  68
step  69
step  70
step  71
step  72
step  73
step  74
step  75
step  76
step  77
step  78
step  79
step  80
step  81
step  82
step  83
step  84
step  85
step  86
step  87
step  88
step  89
step  90
step  91
step  92
step  93
step  94
step  95
step  96
step  97
step  98
step  99
step  100
step  101
step  102
step  103
step  104
step  105
step  106
step  107
step  108
step  109
step  110
step  111
step  112
step  113
step  114
step  115
step  116
step  117
step  118
step  119
step  120
step  121
step  122
step  123
step  124
step  125
step  126
step  127
step  128
step  129
step  130
step  131
step  132
step  133
step  134
step  135
step  136
step  137
step  138
step  139
step  140
step  141
step  142
step  143
step  144
step  145
step  146
step  147
step  148
step  149
step  150
step  151
step  152
st

### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to k−3. Start with the particles on a grid, but with masses derived from a realization of k−3 and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?